In [1]:
/* 27-13 */
-- Create a DMK. 
-- The DMK is encrypted using the password "SQL2014Rocks"
USE master;
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'SQL2014Rocks';
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020

In [2]:
--create our encryptor
USE master;
GO
CREATE CERTIFICATE AW2014BackupCert
   WITH SUBJECT = 'AdventureWorks2014 Backup Encryption Certificate';
GO

--backup the database
USE master;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.270

In [3]:
BACKUP DATABASE AdventureWorks2014
TO DISK = N'C:\Apress\AdventureWorks2014_enc.bak'
WITH
  ENCRYPTION 
   (
   ALGORITHM = AES_256,
   SERVER CERTIFICATE = AW2014BackupCert
   ),
  STATS = 5
GO

Warning: The certificate used for encrypting the database encryption key has not been backed up. You should immediately back up the certificate and the private key associated with the certificate. If the certificate ever becomes unavailable or if you must restore or attach the database on another server, you must have backups of both the certificate and the private key or you will not be able to open the database.

5 percent processed.

10 percent processed.

15 percent processed.

20 percent processed.

...

100 percent processed.

Processed 25744 pages for database 'AdventureWorks2014', file 'AdventureWorks2014_Data' on file 1.

Processed 1 pages for database 'AdventureWorks2014', file 'AdventureWorks2014_Log' on file 1.

BACKUP DATABASE successfully processed 25745 pages in 1.152 seconds (174.588 MB/sec).

Total execution time: 00:00:01.329

In [4]:
-- check the backups
USE msdb;
GO
SELECT bs.database_name,
	   bs.backup_start_date,
	   CASE bs.type
	     WHEN 'D' THEN 'Full Database'
	     WHEN 'I' THEN 'Differential Database' 
	     WHEN 'L' THEN 'Log' 
	     WHEN 'F' THEN 'File or Filegroup' 
	     WHEN 'G' THEN 'Differential File'
	     WHEN 'P' THEN 'Partial'
	     WHEN 'Q' THEN 'Differential Partial'
	     ELSE 'Unknown'
	   END AS BackupType,
	   bmf.physical_device_name,
	   bs.backup_size/1024/1024 as BackSizeMB,
	   bs.encryptor_type, bs.key_algorithm
FROM backupset bs 
INNER JOIN backupmediafamily bmf
ON bs.media_set_id = bmf.media_set_id
WHERE bs.key_algorithm IS NOT NULL
ORDER BY bs.database_name,bs.backup_start_date DESC;
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.047

database_name,backup_start_date,BackupType,physical_device_name,BackSizeMB,encryptor_type,key_algorithm
AdventureWorks2014,2020-10-20 17:23:33.000,Full Database,C:\Apress\AdventureWorks2014_enc.bak,202.08203125000,CERTIFICATE,aes_256
